# Using book ratings to predict similar books to a specific book

Starting with a dataset of Amazon book reviews that has about 3 mil rows. I created a smaller dataset by pulling out only the columns I needed, plus dropping rows that have a null value. 
```
# rating_cols = ['Id', 'User_id', 'Title', 'review/score']
# reviews = pd.read_csv("../datasets/Books_rating.csv", names=rating_cols, usecols=range(4), header=0)

# reviews.dropna(inplace= True)
# reviews.to_csv('../datasets/Book_ratings_clean.csv')
```
to sample a subset randomly `reviews = reviews.sample(frac=0.5, random_state=1).reset_index(drop=True)` 
I didn't need to do this, but it would improve the speed (would also leave out some hits though..)

Next, I updated the column names and imported it into a postgres table so I could easily search it. I'm sure it's just as fast to do it here, but I'm more comfortable with SQL at the moment. 

In [1]:
import pandas as pd
# rating_cols = ['Id', 'User_id', 'Title', 'review/score']
# reviews = pd.read_csv("../datasets/Books_rating.csv", names=rating_cols, usecols=range(4), header=0)

# reviews.dropna(inplace= True)
# reviews.to_csv('../datasets/Book_ratings_clean.csv')

In [2]:
# using that cleaner dataset
ratings = pd.read_csv("../datasets/Book_ratings_clean.csv")
ratings.count()

id         414575
book_id    414575
title      414575
rating     414575
user_id    414575
dtype: int64

many books only have 1 review, aggregating the rating counts and mean to make it easier to work with 

In [3]:
import numpy as np
bookReviews = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
bookReviews.head()


rating       
                                                     size   mean
title                                                           
"A Truthful Impression of the Country": British...      1  70.27
"Beauty Shop-Philly Style"                              2  13.95
"Civilizing" Rio: Reform and Resistance in a Br...      1  32.95
"Come and See" Kids: The Life of Jesus (Catholi...      2   9.65
"Come to Me"                                            1  10.19

Too many rows for my laptop to churn through, so dropping out books with less than 10 reviews

In [4]:
popularBooks = bookReviews['rating']['size'] >= 30
bookReviews[popularBooks].sort_values([('rating', 'mean')], ascending=False)[:15]

rating        
                                                     size    mean
title                                                            
Starkissed                                             35  730.00
The Collected Works of J.G. Frazer: The Golden ...     49  630.00
Biochemistry                                           31  210.51
Stone of Tears (Sword of Truth Series)                386  167.25
Examkrackers MCAT Audio Osmosis with Jordan and...     74  129.50
Just Say No                                            77  119.95
Learning Spanish Like Crazy: Spoken Spanish, Vo...     55  118.49
Out                                                   155  117.95
The Portrait of a Lady (The Classic Collection)        47  112.25
Music & Silence                                        30  110.95
And Never Let Her Go: Thomas Capano, the Deadly...    103  110.95
The Merchant of Venice: William Shakespeare (Ne...     38  110.00
Stranger In A Strange Land: Library Edition           544  108.00
Exploring Psychology, Sixth Edition, in Modules        30  106.66
Little Big Man (Isis)                                  37  104.95

Using the pivot_table function on a DataFrame will construct a user / book rating matrix. This process takes about 5 mins to run on my 2020 MBP, watch the activity monitor to see the process. 
Next, I cast this back to a dataframe to work with, then joined it to the original dataframe to get ratings and user_id's. Left join here leaves out books from the original set that aren't in the popular list (10 or more reviews). 

In [7]:
popularBooksDF = bookReviews[popularBooks]
# bookReviews['title'] = bookReviews['title'].to_string()
popularBooksDF.head()
# popularBooksDF.count()
# join this back with the original ratings df to get the ratings and user_ids
popularBooksDF = popularBooksDF.join(ratings.set_index('title'), on='title', how='left')
popularBooksDF.count()

/var/folders/cl/jrjgywq95pvc95g_k8kn89hw0000gn/T/ipykernel_25588/71587082.py:6: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  popularBooksDF = popularBooksDF.join(ratings.set_index('title'), on='title', how='left')


(rating, size)    191851
(rating, mean)    191851
id                191851
book_id           191851
rating            191851
user_id           191851
dtype: int64

In [8]:
bookReviewsPTable = popularBooksDF.pivot_table(index=['user_id'],columns=['title'],values='rating')
bookReviewsPTable.head()

title,"""More More More,"" Said the Baby Board Book (Caldecott Collection)","1,000 Indian Recipes","1,000 Vegetarian Recipes",10 Days to Faster Reading,100 Samurai Sudoku Puzzles,1001 Books You Must Read Before You Die,1001 Winning Chess Sacrifices and Combinations,101 Things I Wish I Knew When I Got Married: Simple Lessons to Make Love Last,101 Things to Do with a Tortilla,110 People Who Are Screwing Up America (and Al Franken Is #37),...,Zane's Gettin' Buck Wild: Sex Chronicles II,Zane's Skyscraper: A Novel,Zanesville: A Novel,Zen Guitar,Zen Shorts (Caldecott Honor Book),Zen and the Art of Poker: Timeless Secrets to Transform Your Game,Zen in the Martial Arts,Zin! Zin! Zin! A Violin (Aladdin Picture Books),comeback - a mother and daughter's journey through hell and back,the Picture of Dorian Gray
user_id,,,,,,,,,,,,,,,,,,,,,
A00290423P2GEY37XWVAW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A00841771VHT9WNGU0X3J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A00891092QIVH4W1YP46A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A00940571GAOITYS675AR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0099149GMVW6X5BHP2U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
specificBookRatings = bookReviewsPTable["The Picture of Dorian Gray"]
specificBookRatings.count()

577

from the frank kane course: Pandas' corrwith function makes it really easy to compute the pairwise correlation of the book's vector of user rating with every other book! After that, we'll drop any results that have no data, and construct a new DataFrame of books and their correlation score (similarity) to the specific one from above:

In [10]:
similarBooks = bookReviewsPTable.corrwith(specificBookRatings)
similarBooks = similarBooks.dropna()
df = pd.DataFrame(similarBooks)
df.head(10)

/Users/vanessaholland/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/vanessaholland/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,0
title,
A Christmas Carol (Classic Fiction),1.000000
"A Christmas Carol, in Prose: Being a Ghost Story of Christmas (Collected Works of Charles Dickens)",1.000000
A Tale of Two Cities - Literary Touchstone Edition,-1.000000
Alice's Adventures in Wonderland and Through the Looking Glass (Classic Collection (Brilliance Audio)),1.000000
Geoffrey Chaucer (Bloomsbury Poetry Classics),-1.000000
The Jungle Book,-1.000000
The Picture of Dorian Gray,1.000000
The Picture of Dorian Gray (The Classic Collection),0.998289
"The Red Badge of Courage (Lake Illustrated Classics, Collection 1)",1.000000


ToDo- resolve that error. for now it's safe.

In [11]:
similarBooks.sort_values(ascending=False)

title
Alice's Adventures in Wonderland and Through the Looking Glass (Classic Collection (Brilliance Audio))    1.000000
The Picture of Dorian Gray                                                                                1.000000
The Red Badge of Courage (Lake Illustrated Classics, Collection 1)                                        1.000000
Wuthering Heights                                                                                         1.000000
the Picture of Dorian Gray                                                                                1.000000
A Christmas Carol (Classic Fiction)                                                                       1.000000
A Christmas Carol, in Prose: Being a Ghost Story of Christmas (Collected Works of Charles Dickens)        1.000000
The Picture of Dorian Gray (The Classic Collection)                                                       0.998289
A Tale of Two Cities - Literary Touchstone Edition                        

Ten might be too low.. may play around with it. 

In [15]:
df = bookReviews[popularBooks].join(pd.DataFrame(similarBooks, columns=['similarity']))

/var/folders/cl/jrjgywq95pvc95g_k8kn89hw0000gn/T/ipykernel_25588/2821701313.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  df = bookReviews[popularBooks].join(pd.DataFrame(similarBooks, columns=['similarity']))


In [16]:
df.head()

,"(rating, size)","(rating, mean)",similarity
title,,,
"""More More More,"" Said the Baby Board Book (Caldecott Collection)",56,7.19,NaN
"1,000 Indian Recipes",42,7.00,NaN
"1,000 Vegetarian Recipes",40,7.00,NaN
10 Days to Faster Reading,35,7.99,NaN
100 Samurai Sudoku Puzzles,30,9.99,NaN


In [17]:
df.sort_values(['similarity'], ascending=False)[:15]

,"(rating, size)","(rating, mean)",similarity
title,,,
Alice's Adventures in Wonderland and Through the Looking Glass (Classic Collection (Brilliance Audio)),294,18.540000,1.000000
The Picture of Dorian Gray,1184,22.480000,1.000000
"The Red Badge of Courage (Lake Illustrated Classics, Collection 1)",286,14.950000,1.000000
Wuthering Heights,1736,15.975000,1.000000
the Picture of Dorian Gray,592,12.990000,1.000000
A Christmas Carol (Classic Fiction),957,13.980000,1.000000
"A Christmas Carol, in Prose: Being a Ghost Story of Christmas (Collected Works of Charles Dickens)",957,88.940000,1.000000
The Picture of Dorian Gray (The Classic Collection),592,25.040000,0.998289
A Tale of Two Cities - Literary Touchstone Edition,827,5.990000,-1.000000


Follow-ups: remove original book from results, and learn how to work with big data more efficiently 